# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Võ Duy Nhân
- MSSV: 20127056
- Lớp: 20CLC11

# Import

In [1]:
import pandas as pd
import numpy as np
import math
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   
        
def rmse(y, y_hat):
    se = np.mean((y.ravel() - y_hat.ravel())**2)
    return math.sqrt(np.mean(se))

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [4]:
# Phần code cho yêu cầu 1a
(X, y) = (np.array(X_train), np.array(y_train).reshape(-1,1))

lr = OLSLinearRegression().fit(X,y)

y_test_predict = lr.predict(np.array(X_test))

print(lr.get_params())

[[ 1.51013627e-02]
 [ 9.02199807e-02]
 [ 4.29218175e-02]
 [ 1.39289117e-01]
 [-5.67332827e-01]
 [-1.00765115e-04]
 [ 7.40713438e-01]
 [ 1.90935798e-01]
 [ 2.45059736e+01]
 [ 2.39351661e+00]]


In [5]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
rmse(y_test, y_test_predict)

7.064046430584466

Công thức hồi quy

$$\text{Life expectancy} = 0.0151013627 *Adult Morality + 0.0902199807 * BMI + 0.0429218175 * Polio + 0.139289117 * Diphtheria  - 0.567332827 * HIV/AIDS  - 0.000100765115 *  GDP + 0.740713438 * Thinness age 10 - 19 + 0.190935798 * Thinness age 5 - 9 + 24.5059736 * Income composition of resources + 2.39351661 * Schooling$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [6]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

data = np.hstack((np.array(X_train).copy(),np.array(y_train).copy().reshape(-1,1) ))
np.random.shuffle(data)

X_train_clone = data[:, : -1]
y_train_clone = data[:, -1]

    
RMSE_list = np.ones(10)
for i in range(10):
    X_train_feature = X_train_clone[:,i]
    y_train_feature = y_train_clone
    
    RMSE = 0
    for j in range(5):
        X_val = X_train_feature[j * 217 : (j+1) * 217]
        y_val = y_train_feature[j * 217 : (j+1) * 217]
        
        Xtrain_kfold = np.concatenate ((X_train_feature[0 : j * 217],  X_train_feature[(j+1) * 217 : 217 * 5]))
        ytrain_kfold = np.concatenate ((y_train_feature[0 : j * 217], y_train_feature[(j+1) * 217 : 217 * 5]))
        

        lr = OLSLinearRegression().fit(Xtrain_kfold.reshape(-1,1),ytrain_kfold.reshape(-1,1))
        
        y_val_predict = lr.predict(X_val.reshape(-1,1))
        
        RMSE += rmse (y_val, y_val_predict)
    
    RMSE_list[i] = RMSE / 5

print (RMSE_list)


[46.22512429 27.90470836 17.99305181 15.98564663 67.1001855  60.18481154
 51.81247783 51.73478913 13.18601473 11.77609994]


| STT | Mô hình với 1 đặc trưng | RMSE |
|:---:|:-----------------------:|:----:|
|  1  | Adult Mortality         |  46.22512429    |
|  2  | BMI                     |  27.90470836   |
|  3  | Polio                   |  17.99305181    |
|  4  | Diphtheria              |  15.98564663    |
|  5  | HIV/AIDS                |  67.1001855    |
|  6  | GDP                     |  60.18481154    |
|  7  | Thinness age 10-19      |  51.81247783    |
|  8  | Thinness age 5-9        |  51.73478913    |
|  9  | Income composition of resources                     |  13.18601473    |
|  10 | Schooling               |  11.77609994    |

Schooling is the best feature

In [7]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X_train_schooling = np.array(X_train)[:, 9]
y_train_schooling = np.array(y_train)

X_test_schooling = np.array(X_test)[:, 9]
y_test_schooling = np.array(y_test)

lr = OLSLinearRegression().fit(X_train_schooling.reshape(-1,1),y_train_schooling.reshape(-1,1))

y_test_schooling_predict = lr.predict(X_test_schooling.reshape(-1,1))

print(lr.get_params())


[[5.5573994]]


In [8]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
rmse (y_test_schooling_predict, y_test_schooling)

10.260950391655376

Công thức hồi quy

$$\text{Life expectancy} = 5.5573994 * Schooling$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [9]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
data = np.hstack((np.array(X_train).copy(),np.array(y_train).copy().reshape(-1,1) ))
np.random.shuffle(data)

X_train_clone = data[:, : -1]
y_train_clone = data[:, -1]

RMSE_list = np.ones(3)

# Model 1: y =  w1 * Polio + w2 * Diptheria + w3 * HIV/AIDS
X_train_model_1 = np.hstack ((X_train_clone[:,2].reshape(-1,1), X_train_clone[:,3].reshape(-1,1), X_train_clone[:,4].reshape(-1,1)))


RMSE = 0
for j in range(5):
    X_val = X_train_model_1[j * 217 : (j+1) * 217, :]
    y_val = y_train_clone[j * 217 : (j+1) * 217]
    
    Xtrain_kfold = np.concatenate ((X_train_model_1[0 : j * 217, :],  X_train_model_1[(j+1) * 217 : 217 * 5, :]))
    ytrain_kfold = np.concatenate ((y_train_clone[0 : j * 217], y_train_clone[(j+1) * 217 : 217 * 5]))
        

    lr = OLSLinearRegression().fit(Xtrain_kfold,ytrain_kfold.reshape(-1,1))
        
    y_val_predict = lr.predict(X_val)
        
    RMSE += rmse (y_val, y_val_predict)
    
RMSE_list[0] = RMSE / 5

# Model 2:  y = w1 * HIV/AIDS + w2 * Income composition of resources + w3 * Schooling
X_train_model_2 = np.hstack ((X_train_clone[:, 4].reshape(-1,1), X_train_clone[:, 8].reshape(-1,1), X_train_clone[:, 9].reshape(-1,1)))

RMSE = 0
for j in range(5):
    X_val = X_train_model_2[j * 217 : (j+1) * 217, :]
    y_val = y_train_clone[j * 217 : (j+1) * 217]
    
    Xtrain_kfold = np.concatenate ((X_train_model_2[0 : j * 217, :],  X_train_model_2[(j+1) * 217 : 217 * 5, :]))
    ytrain_kfold = np.concatenate ((y_train_clone[0 : j * 217], y_train_clone[(j+1) * 217 : 217 * 5]))
        

    lr = OLSLinearRegression().fit(Xtrain_kfold,ytrain_kfold.reshape(-1,1))
        
    y_val_predict = lr.predict(X_val)
        
    RMSE += rmse (y_val, y_val_predict)
    
RMSE_list[1] = RMSE / 5



# Model 3: y =  w1 * GDP ^ 2
X_train_model_3 = X_train_clone[:,5]**2

RMSE = 0
for j in range(5):
    X_val = X_train_model_3[j * 217 : (j+1) * 217]
    y_val = y_train_clone[j * 217 : (j+1) * 217]
    
    Xtrain_kfold = np.concatenate ((X_train_model_3[0 : j * 217],  X_train_model_3[(j+1) * 217 : 217 * 5]))
    ytrain_kfold = np.concatenate ((y_train_clone[0 : j * 217], y_train_clone[(j+1) * 217 : 217 * 5]))
        

    lr = OLSLinearRegression().fit(Xtrain_kfold.reshape(-1,1),ytrain_kfold.reshape(-1,1))
        
    y_val_predict = lr.predict(X_val.reshape(-1,1))
        
    RMSE += rmse (y_val, y_val_predict)
    
RMSE_list[2] = RMSE / 5


print(RMSE_list)

[15.04847921 11.2842458  66.1290711 ]


| STT | Mô hình | RMSE |
|:---:|:-----------------------:|:----:|
|  1  |  y = w1 * Polio + w2 * Diphtheria + w3 * HIV/AIDS|  15.04847921    |
|  2  | y = w1 * HIV/AIDS + w2 * Income composition of resources + w3 * Schooling                     |  11.2842458   |
|  3  | y = w * $GDP ^ {2}$                   |  66.1290711    |

In [10]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X_train_model_2 = np.hstack ((np.array(X_train)[:, 4].reshape(-1,1), 
                                 np.array(X_train)[:, 8].reshape(-1,1),
                                 np.array(X_train)[:, 9].reshape(-1,1) ))

y_train_model_2 = np.array(y_train)

X_test_model_2 = np.hstack ((np.array(X_test)[:, 4].reshape(-1,1),
                                  np.array(X_test)[:, 8].reshape(-1,1),
                                  np.array(X_test)[:, 9].reshape(-1,1) ))
y_test_model_2 = np.array(y_test)

lr = OLSLinearRegression().fit(X_train_model_2,y_train_model_2.reshape(-1,1))

y_test_model_2_predict = lr.predict(X_test_model_2)

print(lr.get_params())




[[2.69354523e-02]
 [3.51354088e+01]
 [3.72006160e+00]]


In [11]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
rmse(y_test_model_2_predict, y_test_model_2)

9.617618622409767

Công thức hồi quy

$$\text{Life expectancy} = 0.0269354523 * HIV/AIDS + 35.1354088 * Income composition of resources + 3.72006160 * Schooling$$